In [15]:
import pyvista
import lfricflux
import mint
import vtk
import numpy as np

In [5]:
inputFile = '../data/gungho/original/lfric_diag.nc'
configFile = '../configs/lfric.cfg'
# create a flux instance. The geometry is read form the netcdf Ugrid file
lf = lfricflux.LFRicFlux(configFile=configFile, inputFile=inputFile)

In [6]:
# create an empty vector field object whose coordinates are mid-edges
x, y = lf.getEdgeLonLat()
vf = lfricflux.VtkVectors(x, y, vector_field_name='velocity', cartesian=True)

In [7]:
# read the velocity data
u = mint.NcFieldRead(fileName=inputFile, varName='u_in_w2h').data()
v = mint.NcFieldRead(fileName=inputFile, varName='v_in_w2h').data()

In [8]:
# last dimension is number of edges
u.shape

(1, 30, 3072)

In [17]:
# set the vector field
vf.setField(u, v)
# set time = 0, elev = 0
vf.setSlice((0, 0))

In [18]:
def getUVMax(vf):
    vxyz = vf.getFieldArray()
    return np.amax(np.sqrt(np.sum(vxyz**2, axis=1)))

In [19]:
getUVMax(vf)

0.7127091987512543

In [22]:
# create a vector field visualisation
arrow = vtk.vtkArrowSource()
glyphs = vtk.vtkGlyph3D()
glyphs.SetScaleModeToScaleByVector()
glyphs.SetColorModeToColorByVector()
uvMax = getUVMax(vf)
scale = 0.5/ uvMax
glyphs.SetScaleFactor(scale)
glyphs.SetRange(0., uvMax)
glyphs.SetInputData(vf.getMesh())
glyphs.SetSourceConnection(arrow.GetOutputPort())

In [23]:
# add the visualisation to the scene
pyvista.global_theme.cmap = 'jet'
p = pyvista.Plotter(window_size=(900, 800))
p.set_background((0.9, 0.9, 0.92))
p.add_mesh(glyphs)
p.show()

Widget(value="<iframe src='http://localhost:64588/index.html?ui=P_0x182560340_1&reconnect=auto' style='width: …

In [24]:
from ipywidgets import interact
import ipywidgets as widgets

def callbackElev(k):
    # copy the new values
    vf.setSlice((0, k))
    # recompute the max u, v
    uvMax = getUVMax(vf)
    scale = 0.5/ uvMax
    glyphs.SetScaleFactor(scale)
    print(f'elev k = {k}')
    vf.vectorField.Modified()

interact(callbackElev, k=widgets.IntSlider(min=0, max=u.shape[1] - 1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='k', max=29), Output()), _dom_classes=('widget-interact',…

<function __main__.callbackElev(k)>